# Usecase in Glaciology:


The task it to predict glacier ice thickness from all other attributes available in the database is *Glatilda* for [*Glacier Ice Thickness Database*](!https://www.gtn-g.ch/data_catalogue_glathida/).


We will perform the full end-to-end ML workflow to predict ice thickness:
- Data preparation
- Data exploration: look at correlations among attributes (e.g, homework 3)
- Toy problem of a multi variate regression problem: 
    - use all numerical attributes to predict ice thickness.
    - use a regression model of your choice to set up the problem.
    - train, validate, test, discuss performance
    - models can be: Decision trees, RF, SVM, GB(gradient boosting regressor), XGBoost
    - discuss importance feature if you can.

- Perform a hyperparameter search of the chosen regression models. grid search or random.
- Use ensemble learning with Voting or Bagging
- Use ensemble learning with XGBoost (some hyperparameter tuning can be foujnd there: https://www.kaggle.com/code/prashant111/a-guide-on-xgboost-hyperparameters-tuning/notebook). There is an example or randomizedsearch here: https://stackoverflow.com/questions/69786993/tuning-xgboost-hyperparameters-with-randomizedsearchcv
- Be adventurous and try ``auto-sklearn``

In [ ]:
# solution
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import sklearn


In [ ]:
!git clone https://gitlab.com/wgms/glathida.git

In [ ]:
t_path = "glathida/data/glacier.csv"
G = pd.read_csv(t_path, index_col=0)
G.head()